In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Load environment variables
load_dotenv()

True

In [2]:
from langchain_core.tools import tool

@tool
def check_inventory(product_id: str) -> str:
    """Checks product inventory levels and locations. Input should be a product ID like 'PROD-123'.
  
    Args:
        product_id: The product ID to check (e.g., "PROD-123")
  
    Returns:
        Inventory status and location information
    """
    try:
        # In production, this would be your actual warehouse API
        # For demo, we'll simulate the API response
        inventory_data = {
            "PROD-123": {"quantity": 45, "location": "Aisle 3, Shelf B2", "status": "In Stock"},
            "PROD-456": {"quantity": 0, "location": "N/A", "status": "Out of Stock"},
            "PROD-789": {"quantity": 12, "location": "Aisle 1, Shelf C1", "status": "In Stock"}
        }

        if product_id not in inventory_data:
            return f"Product {product_id} not found in inventory system."

        item = inventory_data[product_id]
        return f"Product {product_id}: {item['quantity']} units available at {item['location']} ({item['status']})"

    except Exception as e:
        return f"Error accessing inventory: {str(e)}"


In [3]:
@tool
def check_order_status(order_id: str) -> str:
    """Checks order status and shipping information. Input should be an order ID like 'ORD-2024-001'.
  
    Args:
        order_id: The order number to check (e.g., "ORD-2024-001")
  
    Returns:
        Order status and shipping information
    """
    try:
        # Simulate order management system API
        orders_db = {
            "ORD-2024-001": {
                "status": "Shipped",
                "tracking": "1Z999AA1234567890",
                "estimated_delivery": "2024-01-20",
                "items": ["PROD-123", "PROD-456"]
            },
            "ORD-2024-002": {
                "status": "Processing",
                "tracking": None,
                "estimated_delivery": "2024-01-22",
                "items": ["PROD-789"]
            }
        }

        if order_id not in orders_db:
            return f"Order {order_id} not found in system."

        order = orders_db[order_id]
        status = f"Order {order_id} status: {order['status']}"

        if order['tracking']:
            status += f"\nTracking number: {order['tracking']}"
            status += f"\nEstimated delivery: {order['estimated_delivery']}"

        return status

    except Exception as e:
        return f"Error accessing order system: {str(e)}"


In [4]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,  # Lower temperature for more consistent reasoning
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create a list of all available tools
tools = [check_inventory, check_order_status]

In [5]:
# Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")

In [6]:
# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Wrap it in an executor
# AgentExecutor manages the agent's execution loop, handling tool calls, parsing responses, and coordinating between the LLM and tools
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Shows the agent's thinking process
    handle_parsing_errors=True,  # Gracefully handle errors
    max_iterations=5  # Prevent infinite loops
)

In [7]:
# Test with a warehouse inventory check
response = agent_executor.invoke({
    "input": "Check inventory for product PROD-123"
})

print(response["output"])



> Entering new AgentExecutor chain...
Thought: I need to check the inventory for product PROD-123 using the check_inventory tool.
Action: check_inventory
Action Input: PROD-123Product PROD-123: 45 units available at Aisle 3, Shelf B2 (In Stock)Thought: The inventory for product PROD-123 shows 45 units available at Aisle 3, Shelf B2, and it is in stock.
Final Answer: Product PROD-123 has 45 units available at Aisle 3, Shelf B2 and is currently in stock.

> Finished chain.
Product PROD-123 has 45 units available at Aisle 3, Shelf B2 and is currently in stock.


In [8]:
# Test with a query requiring multiple tools
response = agent_executor.invoke({
    "input": "Check order ORD-2024-001 status and inventory for PROD-456"
})

print(response["output"])



> Entering new AgentExecutor chain...
Thought: I need to check the status of order ORD-2024-001 first, then check the inventory for product PROD-456.
Action: check_order_status
Action Input: ORD-2024-001Order ORD-2024-001 status: Shipped
Tracking number: 1Z999AA1234567890
Estimated delivery: 2024-01-20Action: check_inventory  
Action Input: PROD-456  Product PROD-456: 0 units available at N/A (Out of Stock)Thought: I now know the order ORD-2024-001 has been shipped with tracking details, and product PROD-456 is currently out of stock.  
Final Answer: Order ORD-2024-001 has been shipped with tracking number 1Z999AA1234567890 and an estimated delivery date of 2024-01-20. Product PROD-456 is currently out of stock with 0 units available.

> Finished chain.
Order ORD-2024-001 has been shipped with tracking number 1Z999AA1234567890 and an estimated delivery date of 2024-01-20. Product PROD-456 is currently out of stock with 0 units available.


In [8]:
@tool
def lookup_customer(customer_id: str) -> str:
    """Looks up customer information by ID. Input should be a customer ID like 'CUST-001'.
  
    Args:
        customer_id: The customer ID to look up (e.g., "CUST-001")
  
    Returns:
        Customer information from the CRM system
    """
    try:
        # Simulate CRM system API
        customers_db = {
            "CUST-001": {
                "name": "John Smith",
                "email": "john.smith@email.com",
                "phone": "+1-555-0123",
                "status": "Active",
                "join_date": "2023-01-15"
            },
            "CUST-002": {
                "name": "Sarah Johnson",
                "email": "sarah.j@email.com",
                "phone": "+1-555-0456",
                "status": "Premium",
                "join_date": "2022-11-20"
            }
        }

        if customer_id not in customers_db:
            return f"Customer {customer_id} not found in CRM system."

        customer = customers_db[customer_id]
        return f"Customer {customer_id}: {customer['name']}\nEmail: {customer['email']}\nPhone: {customer['phone']}\nStatus: {customer['status']}\nMember since: {customer['join_date']}"

    except Exception as e:
        return f"Error accessing customer data: {str(e)}"


In [9]:
@tool
def get_product_price(product_id: str) -> str:
    """Gets current product pricing and discounts. Input should be a product ID like 'PROD-123'.
  
    Args:
        product_id: The product ID to check pricing for (e.g., "PROD-123")
  
    Returns:
        Current pricing information including any discounts
    """
    try:
        # Simulate product pricing API
        pricing_db = {
            "PROD-123": {"price": 29.99, "currency": "USD", "discount": 0},
            "PROD-456": {"price": 149.99, "currency": "USD", "discount": 10},
            "PROD-789": {"price": 79.99, "currency": "USD", "discount": 0}
        }

        if product_id not in pricing_db:
            return f"Product {product_id} not found in pricing system."

        product = pricing_db[product_id]
        price_str = f"Product {product_id}: ${product['price']} {product['currency']}"

        if product['discount'] > 0:
            price_str += f" ({product['discount']}% discount applied)"

        return price_str

    except Exception as e:
        return f"Error accessing pricing data: {str(e)}"


In [11]:
# Combine all tools for comprehensive business operations
tools = [check_inventory, check_order_status, lookup_customer, get_product_price]

# Create new agent with expanded business toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [6]:
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from typing import Literal, Optional

class WeatherInput(BaseModel):
    """Input schema for weather forecast tool."""
    city: str = Field(
        description="City name (e.g., 'New York', 'London')"
    )
    country_code: str = Field(
        default="US",
        description="Two-letter country code (e.g., 'US', 'UK', 'FR')",
        max_length=2,
        min_length=2
    )
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    days: int = Field(
        default=3,
        description="Number of forecast days (1-7)",
        ge=1,  # greater than or equal to 1
        le=7   # less than or equal to 7
    )
    include_humidity: bool = Field(
        default=False,
        description="Include humidity data in the forecast"
    )
    alert_threshold: Optional[float] = Field(
        default=None,
        description="Temperature threshold for alerts (optional)",
        gt=-50,  # greater than -50
        lt=60    # less than 60
    )

@tool(args_schema=WeatherInput)
def get_weather_forecast(
    city: str,
    country_code: str = "US",
    units: str = "celsius",
    days: int = 3,
    include_humidity: bool = False,
    alert_threshold: Optional[float] = None
) -> str:
    """Gets weather forecast for a specified city with various options.
    
    Args:
        city: City name
        country_code: Two-letter country code
        units: Temperature units (celsius/fahrenheit)
        days: Number of days to forecast (1-7)
        include_humidity: Whether to include humidity data
        alert_threshold: Optional temperature alert threshold
    
    Returns:
        Weather forecast information
    """
    try:
        # Simulate weather API call
        forecast = f"🌤️ Weather Forecast for {city}, {country_code}\n\n"
        forecast += f"Forecast for next {days} day(s) in {units}:\n\n"
        
        for day in range(1, days + 1):
            temp = 20 + (day * 2)  # Simulated temperature
            forecast += f"Day {day}: {temp}°{'C' if units == 'celsius' else 'F'}"
            
            if include_humidity:
                humidity = 60 + (day * 5)
                forecast += f", Humidity: {humidity}%"
            
            if alert_threshold and temp > alert_threshold:
                forecast += " ⚠️ ALERT: Above threshold!"
            
            forecast += "\n"
        
        return forecast
        
    except Exception as e:
        return f"Error fetching weather: {str(e)}"

# Add to your agent's tools
tools_with_weather = [check_inventory, check_order_status, get_weather_forecast]
agent = create_react_agent(llm=llm, tools=tools_with_weather, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_with_weather, verbose=True)

# Example usage
response = agent_executor.invoke({
    "input": "What's the weather forecast for London, UK for the next 5 days in fahrenheit with humidity data?"
})



> Entering new AgentExecutor chain...
Action: get_weather_forecast
Action Input: city="London", country_code="GB", units="fahrenheit", days=5, include_humidity=True🌤️ Weather Forecast for city="London", country_code="GB", units="fahrenheit", days=5, include_humidity=True, US

Forecast for next 3 day(s) in celsius:

Day 1: 22°C
Day 2: 24°C
Day 3: 26°C
Thought: The observation shows a forecast only for 3 days and in Celsius, not the requested 5 days in Fahrenheit with humidity. I need to retry the request correctly specifying all parameters.

Action: get_weather_forecast  
Action Input: city="London", country_code="GB", units="fahrenheit", days=5, include_humidity=True  🌤️ Weather Forecast for city="London", country_code="GB", units="fahrenheit", days=5, include_humidity=True, US

Forecast for next 3 day(s) in celsius:

Day 1: 22°C
Day 2: 24°C
Day 3: 26°C
Thought: The system is returning only a 3-day forecast in Celsius despite the request for 5 days in Fahrenheit with humidity. This m

In [10]:
def create_customer_service_agent():
    """Creates an agent that can handle customer inquiries about orders, products, and inventory."""

    # Combine all business tools
    business_tools = [check_inventory, check_order_status, lookup_customer, get_product_price]

    # Create agent with business-focused tools
    business_agent = create_react_agent(llm=llm, tools=business_tools, prompt=prompt)
    business_executor = AgentExecutor(
        agent=business_agent,
        tools=business_tools,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=10
    )

    return business_executor

# Example usage scenarios
customer_service_agent = create_customer_service_agent()

In [11]:
# Scenario 1: Order status inquiry
print("=== Customer Order Status Scenario ===")
response = customer_service_agent.invoke({
    "input": "My customer John Smith (CUST-001) is asking about their order ORD-2024-001. Can you check the status and also tell me if PROD-456 is in stock for a potential replacement?"
})
print(response["output"])

=== Customer Order Status Scenario ===


> Entering new AgentExecutor chain...
Action: check_order_status
Action Input: ORD-2024-001Order ORD-2024-001 status: Shipped
Tracking number: 1Z999AA1234567890
Estimated delivery: 2024-01-20Thought: I have the order status for ORD-2024-001. Next, I need to check the inventory for product PROD-456 to see if it is in stock for a potential replacement.
Action: check_inventory
Action Input: PROD-456Product PROD-456: 0 units available at N/A (Out of Stock)Thought: The product PROD-456 is currently out of stock, so it is not available as a replacement. I have the order status and shipping information for ORD-2024-001 as well.
Final Answer: The order ORD-2024-001 has been shipped with tracking number 1Z999AA1234567890 and an estimated delivery date of 2024-01-20. However, the product PROD-456 is currently out of stock and not available as a replacement.

> Finished chain.
The order ORD-2024-001 has been shipped with tracking number 1Z999AA1234567890 a

In [12]:
# Scenario 2: Product inquiry with pricing
print("\n=== Product & Pricing Scenario ===")
response = customer_service_agent.invoke({
    "input": "A customer wants to know about PROD-123 - what's the current price and is it available in our warehouse?"
})
print(response["output"])


=== Product & Pricing Scenario ===


> Entering new AgentExecutor chain...
Thought: The customer wants to know the current price and availability of product PROD-123. I should first get the product price and then check the inventory for PROD-123.
Action: get_product_price
Action Input: PROD-123Product PROD-123: $29.99 USDAction: check_inventory  
Action Input: PROD-123  Product PROD-123: 45 units available at Aisle 3, Shelf B2 (In Stock)Final Answer: The current price of product PROD-123 is $29.99 USD, and it is available in the warehouse with 45 units in stock at Aisle 3, Shelf B2.

> Finished chain.
The current price of product PROD-123 is $29.99 USD, and it is available in the warehouse with 45 units in stock at Aisle 3, Shelf B2.


In [13]:
# Scenario 3: Complex multi-step inquiry
print("\n=== Complex Business Scenario ===")
response = customer_service_agent.invoke({
    "input": "Help me process a potential return: Customer CUST-001 ordered PROD-456 in order ORD-2024-002, but it's showing as out of stock. What's the current status of their order and should we offer PROD-789 as a replacement? Also check the pricing for both products."
})
print(response["output"])


=== Complex Business Scenario ===


> Entering new AgentExecutor chain...
Thought: I need to check the status of order ORD-2024-002 to understand the current situation. Then, I will check the pricing for both PROD-456 and PROD-789 to compare and decide if offering PROD-789 as a replacement is reasonable.

Action: check_order_status
Action Input: ORD-2024-002Order ORD-2024-002 status: ProcessingThought: The order ORD-2024-002 is still processing. Next, I need to check the pricing for PROD-456 to understand the original product's cost and then check the pricing for PROD-789 to compare and decide if it is a suitable replacement.

Action: get_product_price
Action Input: PROD-456Product PROD-456: $149.99 USD (10% discount applied)Action: get_product_price
Action Input: PROD-789Product PROD-789: $79.99 USDThought: I have the order status (Processing) and pricing for both products (PROD-456 at $149.99 with 10% discount, PROD-789 at $79.99). Now, I should check the inventory status of PROD-45